# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [70]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [71]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [72]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()
print(train_num)

891


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [73]:
df.isnull().sum().sort_values(ascending=False)

Age       263
Fare        1
Parch       0
SibSp       0
Pclass      0
dtype: int64

In [74]:
print(df[df['Age'].isnull()])

     Pclass  Age  SibSp  Parch     Fare
5         3  NaN      0      0   8.4583
17        2  NaN      0      0  13.0000
19        3  NaN      0      0   7.2250
26        3  NaN      0      0   7.2250
28        3  NaN      0      0   7.8792
..      ...  ...    ...    ...      ...
408       3  NaN      0      0   7.7208
410       3  NaN      0      0   7.7500
413       3  NaN      0      0   8.0500
416       3  NaN      0      0   8.0500
417       3  NaN      1      1  22.3583

[263 rows x 5 columns]


In [5]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6960299128976762

In [7]:
"""
Your Code Here
"""
df_0 = df.fillna(0)
train_X = df_0[:train_num]
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6971535084032942

In [80]:
df_m = df
df_m['Age'] = df['Age'].fillna(value=df.groupby('Pclass')['Age'].transform('mean'))
df_m = df_m.fillna(0)
train_X = df_m[:train_num]
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7015532378864078

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [35]:
"""
Your Code Here
"""
train_X = StandardScaler().fit_transform(df_m[:train_num])
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6959350475762885

In [37]:
train_X = StandardScaler().fit_transform(df_0[:train_num])
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6982582017719778

In [38]:
train_X = StandardScaler().fit_transform(df_m1[:train_num])
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6982582017719778

In [36]:
train_X = MinMaxScaler().fit_transform(df_m[:train_num])
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7004675175820018

In [39]:
train_X = MinMaxScaler().fit_transform(df_m1[:train_num])
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [56]:
train_X = MinMaxScaler().fit_transform(df_0[:train_num])
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.707234411677194

In [44]:
df[df['Age']==2]

,Pclass,Age,SibSp,Parch,Fare


In [61]:
df['Age'].value_counts()

24.00    47
22.00    43
21.00    41
30.00    40
18.00    39
         ..
76.00     1
22.50     1
74.00     1
80.00     1
0.67      1
Name: Age, Length: 98, dtype: int64

In [79]:
df_1 = df.fillna(1)
train_X = MinMaxScaler().fit_transform(df_1[:train_num])
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7083580071828119

最小最大化配合補0的效果最好，原因的話目前不清楚。